# Retrieve CO-OPS data from an arbitrary time interval and region

In [ ]:
import searvey

In [ ]:
import geopandas
from matplotlib import pyplot

countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# retrieve a [list of CO-OPS stations](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_stations)

In [ ]:
from searvey.coops import get_coops_stations

all_stations = get_coops_stations(metadata_source='main')
all_stations

In [ ]:
figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

all_stations.drop_duplicates().plot(ax=axis, column='status', legend=True)
countries.plot(color='lightgrey', ax=axis, zorder=-1)

axis.set_title(f'all CO-OPS stations')

# retrieve a [list of CO-OPS stations within an arbitrary polygon](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_stations_within_region)

In [ ]:
import shapely

# This is the actualBBox of the east coast, but this takes too long to retrieve.
# Since we execute this notebook on CI, let's actually use Florida
#east_coast = shapely.geometry.box(-85, 25, -65, 45)
east_coast = shapely.geometry.box(-85, 25, -65, 30)
east_coast_stations = get_coops_stations(metadata_source='main', region=east_coast)
east_coast_stations

In [ ]:
figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

east_coast_stations.plot(ax=axis, column='status', legend=True)

xlim = axis.get_xlim()
ylim = axis.get_ylim()
countries.plot(color='lightgrey', ax=axis, zorder=-1)
axis.set_xlim(xlim)
axis.set_ylim(ylim)

axis.set_title(f'CO-OPS stations on the U.S. East Coast')

# retrieve a CO-OPS data product from a [specific station](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.COOPS_Station)

In [ ]:
import pytz
from datetime import datetime
from searvey._coops_api import fetch_coops_station

station_id = '1612480'
station_water_levels = fetch_coops_station(
    station_id=station_id,
    start_date=datetime(2022, 4, 2, 12, tzinfo=pytz.utc),
    end_date=datetime(2022, 4, 2, 12, 30, tzinfo=pytz.utc),
    product='water_level',
)
station_water_levels

In [ ]:
import pandas as pd

figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

axis.errorbar(
    station_water_levels.index,
    station_water_levels['value'],
    yerr=station_water_levels['sigma'].values,
)

start_date = pd.to_datetime(station_water_levels.index.min())
end_date = pd.to_datetime(station_water_levels.index.max())
axis.set_title(f'water level observed by CO-OPS station {station_id} between {start_date} and {end_date}')

# retrieve a CO-OPS data product from a lsit of stations

In [ ]:
from multifutures import multiprocess

stations = east_coast_stations[(east_coast_stations.status == 'active') & (east_coast_stations.station_type == 'waterlevels')]
station_water_levels = pd.concat(
    [
        result.result for result in multiprocess(
            fetch_coops_station,
            [
                {
                    'station_id': nos_id,
                    'start_date': datetime(2022, 4, 2, 12, tzinfo=pytz.utc),
                    'end_date': datetime(2022, 4, 2, 12, 30, tzinfo=pytz.utc),
                    'product': 'water_level',
                } for nos_id, _ in stations.iterrows()
            ],
        ) 
    ],
    keys=[nos_id for nos_id, _ in stations.iterrows()],
    names=['nos_id', 'time']
)
station_water_levels

# additional documentation
## https://searvey.readthedocs.io
## https://github.com/oceanmodeling/searvey
